In this notebook we show how to prepare all the input and parameter files necessary to run Diva4D.<br>
The code rely on the classes defined for 2D and 4D cases.

In [13]:
import os
import logging
import shutil
import subprocess
import diva4D_classes
import diva_classes
import importlib

Reload the module (only during code development), uncomment if necessary.

In [20]:
importlib.reload(diva4D_classes)

<module 'diva4D_classes' from '/home/ctroupin/Projects/SeaDataCloud/Python/diva4D_classes.py'>

# User inputs and configuration

Indicate the path to the Diva installation you want to work with (diva-x.y.z).

In [21]:
divabasedir = "/home/ctroupin/Software/DIVA/diva-4.7.1"

## Create the sub-directories and files

In [22]:
divadirs = diva_classes.DivaDirectories(divabasedir)

INFO:diva2D:Diva binary directory: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/bin
INFO:diva2D:Diva source directory: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/src/Fortan
INFO:diva2D:Diva 2D directory: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped
INFO:diva2D:Diva 4D directory: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology
INFO:diva2D:Diva 4D input directory: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/input


In [23]:
diva2Dfiles = diva_classes.Diva2Dfiles(divadirs.diva2ddir)

INFO:diva2D:Contour file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/coast.cont
INFO:diva2D:Parameter file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/param.par
INFO:diva2D:Data file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/data.dat
INFO:diva2D:Valatxy file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/valatxy.coord
INFO:diva2D:Result file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/output/ghertonetcdf/results.nc
INFO:diva2D:Mesh file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/meshgenwork/fort.22
INFO:diva2D:Mesh topo file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/meshgenwork/fort.23


In [24]:
diva4Dfiles = diva4D_classes.Diva4Dfiles(divadirs.diva4ddir)

INFO:diva4D_classes:datasource file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/datasource
INFO:diva4D_classes:constandrefe file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/constandrefe
INFO:diva4D_classes:driver file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/driver
INFO:diva4D_classes:monthlist file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/monthlist
INFO:diva4D_classes:qflist file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/qflist
INFO:diva4D_classes:varlist file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/varlist
INFO:diva4D_classes:yearlist file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/yearlist
INFO:diva4D_classes:contourdepth file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/contourdepth
INFO:diva4D_classes:ncdfinfo file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/ncdfinfo
INFO:diva4D_classes:param.par file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA

## List of user parameters

Set the different parameters necessary for a 4D analysis.<br>
For clarity, they are separated by input files.

### List of ODV spreadsheet files

In [25]:
datasource = diva4D_classes.Datasource('./blacksea_data_CTD.txt')

### Parameters for the advection constraint and the reference field

* advection_flag: Flag that indicates if advection is activated
* ref_flag: Flag that indicates if there is a reference field
* var_year_code: Variable year code
* var_month_code: Variable month code

In [26]:
advection_flag, ref_flag, var_year_code, var_month_code = 0, 0, '00000000', '0000'
constandrefe = diva4D_classes.Constandrefe(advection_flag, ref_flag, var_year_code, var_month_code)

### Parameters for the 4D analysis

* extraction_flag: Data extraction: 1 do it, 0 do nothing, -1 press coord, -10 pressure+Saunders
* coast_flag: boundary lines and coastlines generation: 0 nothing, 1: contours, 2: UV, 3: 1+2
* clean_flag: cleaning data on mesh: 1, 2: RL, 3: both, 4: 1 + outliers elimination, 5: =4+2
* min_datanum: minimal number of data in a layer. If less, uses data from any month.
* param_flag: Parameters estimation and vertical filtering:
* min_l, max_l: minimal/maximal correlation length
* min_snr, max_snr: minimal/maximal signal-to-noise ratio
* analysisref_flag: Analysis and reference field
* lower_level, upper_level: lower/upper level numbers
* netcdf4d_flag: 4D netcdf files generation
* gnuplot_flag: gnuplot plots: 0 or 1
* detrend_groupnum: Data detrending: number of groups, 0 if no detrending.

In [27]:
extraction_flag, coast_flag, clean_flag, min_datanum, param_flag, min_l, max_l, min_snr, max_snr,\
analysisref_flag, lower_level, upper_level, netcdf4d_flag, gnuplot_flag, detrend_groupnum =\
1, 1, 0, 0, -30, 1.4, 10., 0.1, 5.0, 1, 1, 3, 0, 0, 0

In [28]:
driver = diva4D_classes.Driver(extraction_flag, coast_flag, clean_flag, min_datanum, param_flag, min_l,\
                               max_l, min_snr, max_snr, analysisref_flag, lower_level,\
                               upper_level, netcdf4d_flag, gnuplot_flag, detrend_groupnum)

### Parameters for the netCDF file

* ncdf_title: Title string for 3D NetCDF file
* ncdf_reftime: Reference time for data (ie: days since since 1900-01-01), if not climatological data
* ncdf_timeval: Time value (that reprsents the data set), if not climatological data
* ncdf_cellmethod: Cell_methode string:
* ncdf_institution: Institution name: where the dataset was produced.
* ncdf_groupemail: Production group and e-mail
* ncdf_source: Source (observation, radiosonde, database, model-generated data,...)
* ndf_comment: Comment
* ncdf_authoremail: Autor e-mail adress (or contact person to report problems)
* ncdf_acknowlegment: Acknowledgement

In [29]:
ncdf_title, ncdf_reftime, ncdf_timeval, ncdf_cellmethod,\
ncdf_institution, ncdf_groupemail, ncdf_source,\
ncdf_comment, ncdf_authoremail, ncdf_acknowlegment =\
'4D Test on the Black Sea', 'months since since xxxx-01-01', 1200, 'time: mean (this month data from all years)',\
'University of Liege, GeoHydrodynamics and Environment Research',\
'Diva group. E-mails : a.barth@ulg.ac.be ; swatelet@ulg.ac.be',\
'data_from various sources for diva software testing work', 'No comment', 'swatelet@ulg.ac.be',\
'No acknowledgement'

In [30]:
netcdfinfo = diva4D_classes.Ncdfinfo(ncdf_title, ncdf_reftime, ncdf_timeval, ncdf_cellmethod,\
                                     ncdf_institution, ncdf_groupemail, ncdf_source,\
                                     ncdf_comment, ncdf_authoremail, ncdf_acknowlegment)

### Parameters for the output grid

* xmin, ymin: longitute and latitude of the lower-left corner of the domain
* nx, ny: number of cells in the x and y directions
* dx, dy: spatial steps in the x and y directions

In [31]:
xmin, ymin, nx, ny, dx, dy = 27.0, 40.0, 151, 76, 0.1, 0.1

### Parameters for the analysis

In [32]:
CorrelationLength = 1.5
SignalToNoiseRatio = 0.5
VarianceBackgroundField = 1.0
ExclusionValue = -99.
iCoordChange = 2
iSpec = 11
iReg = 2

In [33]:
parameters2D = diva_classes.Diva2DParameters(CorrelationLength, iCoordChange, iSpec, iReg, xmin, ymin, dx, dy, nx, ny,
                                ExclusionValue, SignalToNoiseRatio, VarianceBackgroundField)

## Others: monthslist, QC flags, variables, years, contour levels

In [34]:
months = diva4D_classes.Monthlist(['0101', '0202', '0303'])
qflags = diva4D_classes.Qflist([0, 1])
variables = diva4D_classes.Varlist(['Temperature', 'Salinity'])
years = diva4D_classes.Yearlist(['19002009',])
contours = diva4D_classes.Contourdepth([30, 20, 10, 0])

# Create the new input files

## Write the different input files using the created objects

In [35]:
datasource.write_to(diva4Dfiles.datasource)

INFO:diva4D_classes:Written into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/datasource


In [36]:
constandrefe.write_to(diva4Dfiles.constandrefe)

INFO:diva4D_classes:File into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/constandrefe


In [37]:
driver.write_to(diva4Dfiles.driver)

INFO:diva4D_classes:Written into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/driver


In [38]:
months.write_to(diva4Dfiles.monthlist)

INFO:diva4D_classes:Written in file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/monthlist


In [39]:
qflags.write_to(diva4Dfiles.qflist)

INFO:diva4D_classes:Written in file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/qflist


In [40]:
variables.write_to(diva4Dfiles.varlist)

INFO:diva4D_classes:Written in file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/varlist


In [41]:
years.write_to(diva4Dfiles.yearlist)

INFO:diva4D_classes:Written into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/yearlist


In [42]:
netcdfinfo.write_to(diva4Dfiles.ncdfinfo)

INFO:diva4D_classes:Written into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/ncdfinfo


In [43]:
parameters2D.write_to_file(diva4Dfiles.param)

# Run divadoall (4D analysis)

In [44]:
divadoall_script = './divadoall'
env = os.environ.copy()
env["PATH"] = ".:" + env["PATH"]

p = subprocess.Popen(divadoall_script, stdout=subprocess.PIPE, cwd=divadirs.diva4ddir, env=env)
out = p.stdout.read()

print(out)

b"divadoall: divadoall run starts at:\nFri Feb 10 16:07:42 CET 2017\ndivadoall:  Start divadoall: idorefe = 0\ndivadoall:                   idoanal = 1\ndivadoall: Working in /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology\ndivadoall: Cleaning up the old NC info and list files\ndivadoall: variable Temperature\ndivadoall: year 19002009\ndivadoall: Erasing 3DNCD list\ndivadoall: Erasing 3DNCD info\ndivadoall: variable Salinity\ndivadoall: year 19002009\ndivadoall: Erasing 3DNCD list\ndivadoall: Erasing 3DNCD info\ndivadoall: Cleaning up the old data files\ndivadoall: variable Temperature\ndivadoall: year 19002009\ndivadoall: month 0101\ndivadoall: month 0202\ndivadoall: month 0303\ndivadoall: variable Salinity\ndivadoall: year 19002009\ndivadoall: month 0101\ndivadoall: month 0202\ndivadoall: month 0303\ndivadoall: Data source file is ./blacksea_data_CTD.txt\ndivadoall: Trying to guess ODV format on ./blacksea_data_CTD.txt\n2331776 and 145736\n11, 16\nNo depth axis (or speed+dep